In [ ]:
# SIMPLE PERLIN NOISE MOVING BALL
import tkinter as tk
import math
from perlin_noise import PerlinNoise

# ==== Vektor 2D sederhana ====
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def add(self, other):
        self.x += other.x
        self.y += other.y

# ==== Objek Bola ====
class Ball:
    def __init__(self, width, height):
        self.pos = Vector(width / 2, height / 2)  # Mulai dari tengah layar
        self.speed = 2                            # Kecepatan gerak
        self.noise = PerlinNoise()                # Generator Perlin Noise
        self.t = 0                                # Waktu untuk noise

    def update(self):
        # Ambil nilai noise (antara -1 sampai 1)
        noise_val = self.noise(self.t)
        angle = noise_val * 2 * math.pi  # Ubah jadi sudut arah (0–360 derajat)

        # Hitung arah gerak berdasarkan sudut
        dx = math.cos(angle) * self.speed
        dy = math.sin(angle) * self.speed
        self.pos.add(Vector(dx, dy))  # Tambahkan arah ke posisi

        self.t += 0.01  # Geser waktu untuk noise agar arah berubah perlahan

    def display(self, canvas):
        x = self.pos.x
        y = self.pos.y
        r = 10  # Radius bola

        # Gambar lingkaran (bola)
        canvas.create_oval(x - r, y - r, x + r, y + r, fill='orange', outline='black')

# ==== Program Utama ====
WIDTH = 400
HEIGHT = 300

root = tk.Tk()
root.title("Bola Bergerak dengan Perlin Noise")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg='white')
canvas.pack()

ball = Ball(WIDTH, HEIGHT)

def animate():
    canvas.delete("all")
    ball.update()
    ball.display(canvas)
    root.after(33, animate)  # ~30 fps

animate()
root.mainloop()


In [ ]:
# SIMPLE PERLIN NOISE MOVING TRIANGLE NO BORDER
import tkinter as tk
import math
from perlin_noise import PerlinNoise

# ==== Vektor 2D sederhana ====
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def add(self, other):
        self.x += other.x
        self.y += other.y

# ==== Objek Segitiga ====
class Triangle:
    def __init__(self, width, height):
        self.pos = Vector(width / 2, height / 2)  # Mulai dari tengah layar
        self.angle = 0                            # Sudut rotasi awal
        self.speed = 2                            # Kecepatan tetap
        self.noise = PerlinNoise()                # Buat objek noise
        self.t = 0                                # Waktu untuk perlin noise

    def update(self):
        # Gunakan perlin noise untuk menentukan arah gerak (sudut)
        noise_value = self.noise(self.t)          # Ambil nilai noise (antara -1 sampai 1)
        self.angle = noise_value * 2 * math.pi    # Konversi ke sudut (0–360 derajat)
        self.t += 0.01                            # Geser waktu sedikit agar gerak berubah

        # Ubah arah menjadi vektor gerak
        dx = math.cos(self.angle) * self.speed
        dy = math.sin(self.angle) * self.speed
        self.pos.add(Vector(dx, dy))              # Tambahkan gerakan ke posisi

    def display(self, canvas):
        x = self.pos.x
        y = self.pos.y
        r = 10  # Ukuran segitiga

        # Hitung 3 titik segitiga berdasarkan posisi dan arah (angle)
        points = [
            (x + math.sin(self.angle) * -r, y - math.cos(self.angle) * -r),
            (x + math.sin(self.angle + math.pi * 2 / 3) * r, y - math.cos(self.angle + math.pi * 2 / 3) * r),
            (x + math.sin(self.angle - math.pi * 2 / 3) * r, y - math.cos(self.angle - math.pi * 2 / 3) * r),
        ]

        canvas.create_polygon(points, fill='lightblue', outline='black')

# ==== Main Program ====
WIDTH = 400
HEIGHT = 300

root = tk.Tk()
root.title("Segitiga Bergerak dengan Perlin Noise")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg='white')
canvas.pack()

triangle = Triangle(WIDTH, HEIGHT)

def animate():
    canvas.delete("all")
    triangle.update()
    triangle.display(canvas)
    root.after(33, animate)  # ~30 fps

animate()
root.mainloop()


In [ ]:
# SIMPLE PERLIN NOISE MOVING TRIANGLE WITH BORDER
import tkinter as tk
import math
from perlin_noise import PerlinNoise  # pip install perlin-noise

# ================= Vector =================
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def add(self, v):
        self.x += v.x
        self.y += v.y

    def sub(self, v):
        self.x -= v.x
        self.y -= v.y

    def mult(self, n):
        self.x *= n
        self.y *= n

    def div(self, n):
        if n != 0:
            self.x /= n
            self.y /= n

    def mag(self):
        return math.sqrt(self.x**2 + self.y**2)

    def normalize(self):
        m = self.mag()
        if m != 0:
            self.div(m)

    def limit(self, max_val):
        if self.mag() > max_val:
            self.normalize()
            self.mult(max_val)

    def heading(self):
        return math.atan2(self.y, self.x)

    def copy(self):
        return Vector(self.x, self.y)

    @staticmethod
    def sub_vec(v1, v2):
        return Vector(v1.x - v2.x, v1.y - v2.y)

# ================= FlowField =================
class FlowField:
    def __init__(self, resolution, width, height):
        self.resolution = resolution
        self.cols = width // resolution
        self.rows = height // resolution
        self.field = [[Vector() for _ in range(self.rows)] for _ in range(self.cols)]
        self.noise = PerlinNoise(octaves=2)
        self.init_field()

    def init_field(self):
        for i in range(self.cols):
            for j in range(self.rows):
                angle = self.noise([i * 0.1, j * 0.1]) * 2 * math.pi
                self.field[i][j] = Vector(math.cos(angle), math.sin(angle))

    def lookup(self, position):
        col = int(position.x // self.resolution)
        row = int(position.y // self.resolution)
        col = max(0, min(col, self.cols - 1))
        row = max(0, min(row, self.rows - 1))
        return self.field[col][row].copy()

# ================= Vehicle =================
class Vehicle:
    def __init__(self, x, y, width, height):
        self.position = Vector(x, y)
        self.velocity = Vector()
        self.acceleration = Vector()
        self.maxspeed = 4
        self.maxforce = 0.2
        self.r = 6
        self.w = width
        self.h = height

    def apply_force(self, force):
        self.acceleration.add(force)

    def follow(self, flowfield):
        desired = flowfield.lookup(self.position)
        desired.mult(self.maxspeed)
        steer = Vector.sub_vec(desired, self.velocity)
        steer.limit(self.maxforce)
        self.apply_force(steer)

    def update(self):
        self.velocity.add(self.acceleration)
        self.velocity.limit(self.maxspeed)
        self.position.add(self.velocity)
        self.acceleration.mult(0)

    def borders(self):
        if self.position.x < -self.r:
            self.position.x = self.w + self.r
        if self.position.y < -self.r:
            self.position.y = self.h + self.r
        if self.position.x > self.w + self.r:
            self.position.x = -self.r
        if self.position.y > self.h + self.r:
            self.position.y = -self.r

    def display(self, canvas):
        theta = self.velocity.heading() + math.pi / 2
        x, y, r = self.position.x, self.position.y, self.r
        points = [
            (x + math.sin(theta) * -r * 2, y - math.cos(theta) * -r * 2),
            (x + math.sin(theta + 2 * math.pi / 3) * r * 2, y - math.cos(theta + 2 * math.pi / 3) * r * 2),
            (x + math.sin(theta - 2 * math.pi / 3) * r * 2, y - math.cos(theta - 2 * math.pi / 3) * r * 2)
        ]
        canvas.create_polygon(points, fill="lightblue", outline="black")

# ================= Main App =================
class FlowFieldApp:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.root = tk.Tk()
        self.root.title("1 Segitiga Mengikuti Flow Field")
        self.canvas = tk.Canvas(self.root, width=self.width, height=self.height, bg="white")
        self.canvas.pack()

        self.flowfield = FlowField(20, self.width, self.height)
        self.vehicle = Vehicle(self.width / 2, self.height / 2, self.width, self.height)

        self.draw()

        self.root.mainloop()

    def draw(self):
        self.canvas.delete("all")
        self.vehicle.follow(self.flowfield)
        self.vehicle.update()
        self.vehicle.borders()
        self.vehicle.display(self.canvas)
        self.canvas.after(33, self.draw)  # ~30 FPS

# ================= Jalankan Program =================
if __name__ == "__main__":
    app = FlowFieldApp(360, 240)


In [ ]:
import tkinter as tk
import math

# ===== Vector class =====
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def add(self, v):
        self.x += v.x
        self.y += v.y

    def sub(self, v):
        self.x -= v.x
        self.y -= v.y

    def mult(self, n):
        self.x *= n
        self.y *= n

    def div(self, n):
        if n != 0:
            self.x /= n
            self.y /= n

    def mag(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)

    def normalize(self):
        m = self.mag()
        if m != 0:
            self.div(m)

    def limit(self, max_val):
        m = self.mag()
        if m > max_val:
            self.normalize()
            self.mult(max_val)

    def copy(self):
        return Vector(self.x, self.y)

    @staticmethod
    def sub_vec(v1, v2):
        return Vector(v1.x - v2.x, v1.y - v2.y)

# ===== Bola mengejar mouse =====
class Vehicle:
    def __init__(self, x, y):
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.maxspeed = 5
        self.maxforce = 0.2

    def apply_force(self, force):
        # Menambahkan gaya ke percepatan
        self.acceleration.add(force)

    def seek(self, target):
        """Mengubah arah bola untuk mengejar target (mouse)"""
        # desired = mouse - posisi
        desired = Vector.sub_vec(target, self.position)
        # normalisasi desired
        desired.normalize()
        # kalikan dengan kecepatan maksimum
        desired.mult(self.maxspeed)
        # steer = desired - velocity
        steer = Vector.sub_vec(desired, self.velocity)
        # Batasi kecepatan maksimum
        steer.limit(self.maxforce)
        # Terapkan gaya steer
        self.apply_force(steer)

    def update(self):
        """Update posisi bola berdasarkan percepatan dan kecepatan"""
        # Tambahkan percepatan ke kecepatan
        self.velocity.add(self.acceleration)
        # Batasi kecepatan maksimum
        self.velocity.limit(self.maxspeed)
        # Tambahkan kecepatan ke posisi
        self.position.add(self.velocity)
        # Reset percepatan
        # untuk menghindari penumpukan gaya
        # agar bola tidak bergerak terlalu cepat
        self.acceleration.mult(0)

    def display(self, canvas):
        r = 10
        canvas.create_oval(self.position.x - r, self.position.y - r,
                           self.position.x + r, self.position.y + r,
                           fill="lightblue", outline="black")

# ===== Tkinter Setup =====
WIDTH, HEIGHT = 400, 300
root = tk.Tk()
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

vehicle = Vehicle(WIDTH / 2, HEIGHT / 2)
mouse = Vector(WIDTH / 2, HEIGHT / 2)

def mouse_motion(event):
    mouse.x = event.x
    mouse.y = event.y

canvas.bind("<Motion>", mouse_motion)

def draw():
    canvas.delete("all")
    vehicle.seek(mouse)
    vehicle.update()
    vehicle.display(canvas)

    # Gambar target
    canvas.create_oval(mouse.x - 3, mouse.y - 3, mouse.x + 3, mouse.y + 3, fill="red")

    root.after(30, draw)

draw()
root.mainloop()


In [ ]:
import tkinter as tk
import math

# ===== Vector Class =====
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def add(self, v):
        self.x += v.x
        self.y += v.y

    def sub(self, v):
        self.x -= v.x
        self.y -= v.y

    def mult(self, n):
        self.x *= n
        self.y *= n

    def div(self, n):
        if n != 0:
            self.x /= n
            self.y /= n

    def mag(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)

    def normalize(self):
        m = self.mag()
        if m != 0:
            self.div(m)

    def limit(self, max_val):
        m = self.mag()
        if m > max_val:
            self.normalize()
            self.mult(max_val)

    def copy(self):
        return Vector(self.x, self.y)

    @staticmethod
    def sub_vec(v1, v2):
        return Vector(v1.x - v2.x, v1.y - v2.y)

# ===== Kendaraan yang Mengejar Mouse =====
class Vehicle:
    def __init__(self, x, y):
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.maxspeed = 5
        self.maxforce = 0.2
        self.last_steer = Vector()
        self.last_desired = Vector()

    def apply_force(self, force):
        self.acceleration.add(force)

    def seek(self, target):
        desired = Vector.sub_vec(target, self.position)
        desired.normalize()
        desired.mult(self.maxspeed)

        steer = Vector.sub_vec(desired, self.velocity)
        steer.limit(self.maxforce)

        self.last_steer = steer.copy()
        self.last_desired = desired.copy()
        self.apply_force(steer)

        return desired, self.velocity.copy(), steer

    def update(self):
        self.velocity.add(self.acceleration)
        self.velocity.limit(self.maxspeed)
        self.position.add(self.velocity)
        self.acceleration.mult(0)

    def display(self, canvas):
        r = 10
        canvas.create_oval(self.position.x - r, self.position.y - r,
                           self.position.x + r, self.position.y + r,
                           fill="lightblue", outline="black")

    def draw_vector(self, canvas, vec, color, label):
        x1, y1 = self.position.x, self.position.y
        x2, y2 = x1 + vec.x * 10, y1 + vec.y * 10
        canvas.create_line(x1, y1, x2, y2, fill=color, width=2, arrow=tk.LAST)
        canvas.create_text(x2 + 10, y2, text=label, fill=color, anchor='w')

    def draw_calculation_text(self, canvas, x, y):
        canvas.create_text(x, y, anchor="nw", fill="black", font=("Arial", 10), text=
            f"Position     = ({self.position.x:.2f}, {self.position.y:.2f})\n"
            f"Mouse        = ({mouse.x:.2f}, {mouse.y:.2f})\n"
            f"Desired      = ({self.last_desired.x:.2f}, {self.last_desired.y:.2f})\n"
            f"Velocity     = ({self.velocity.x:.2f}, {self.velocity.y:.2f})\n"
            f"Steer        = ({self.last_steer.x:.2f}, {self.last_steer.y:.2f})"
        )

# ===== Tkinter Setup =====
WIDTH, HEIGHT = 400, 350
root = tk.Tk()
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

vehicle = Vehicle(WIDTH / 2, HEIGHT / 2)
mouse = Vector(WIDTH / 2, HEIGHT / 2)

def mouse_motion(event):
    mouse.x = event.x
    mouse.y = event.y

canvas.bind("<Motion>", mouse_motion)

def draw():
    canvas.delete("all")

    # Target mouse (titik merah)
    canvas.create_oval(mouse.x - 3, mouse.y - 3, mouse.x + 3, mouse.y + 3, fill="red")

    # Hitung dan update
    desired, velocity, steer = vehicle.seek(mouse)
    vehicle.update()
    vehicle.display(canvas)

    # Gambar vektor
    vehicle.draw_vector(canvas, desired, "blue", "desired")
    vehicle.draw_vector(canvas, velocity, "green", "velocity")
    vehicle.draw_vector(canvas, steer, "red", "steer")

    # Gambar perhitungan teks di bawah
    vehicle.draw_calculation_text(canvas, 10, HEIGHT - 90)

    root.after(30, draw)

draw()
root.mainloop()


In [ ]:
#SCRIPT18
#PERLIN NOISE MOVING TRIANGLE WITH BORDER
import tkinter as tk
import math
import random
from perlin_noise import PerlinNoise

class Vector:
    def __init__(self, x=0, y=0):
        self.x = x  # Koordinat x
        self.y = y  # Koordinat y

    def add(self, v):  # Penjumlahan vektor
        self.x += v.x
        self.y += v.y

    def sub(self, v):  # Pengurangan vektor
        self.x -= v.x
        self.y -= v.y

    def mult(self, n):  # Perkalian skalar
        self.x *= n
        self.y *= n

    def div(self, n):  # Pembagian skalar
        if n != 0:
            self.x /= n
            self.y /= n

    def mag(self):  # Menghitung besar vektor
        return math.sqrt(self.x**2 + self.y**2)

    def limit(self, max_val):  # Membatasi besar vektor
        m = self.mag()
        if m > max_val:
            self.normalize()
            self.mult(max_val)

    def normalize(self):  # Membuat vektor menjadi satuan (panjang=1)
        m = self.mag()
        if m != 0:
            self.div(m)

    def copy(self):  # Membuat salinan vektor
        return Vector(self.x, self.y)

    def heading(self):  # Menghitung sudut vektor
        return math.atan2(self.y, self.x)

    @staticmethod
    def sub_vec(v1, v2):  # Static method untuk pengurangan vektor
        return Vector(v1.x - v2.x, v1.y - v2.y)


# ========== FlowField ==========
class FlowField:
    def __init__(self, resolution, width, height):
        self.resolution = resolution  # Ukuran grid
        self.cols = width // resolution  # Jumlah kolom
        self.rows = height // resolution  # Jumlah baris
        # Inisialisasi grid vektor
        self.field = [[Vector() for _ in range(self.rows)] for _ in range(self.cols)]
        self.noise = PerlinNoise(octaves=2)  # Generator noise
        self.init_field()  # Inisialisasi medan

    def init_field(self):
        # Mengisi grid dengan vektor berdasarkan Perlin Noise
        for i in range(self.cols):
            for j in range(self.rows):
                angle = self.noise([i * 0.1, j * 0.1]) * 2 * math.pi
                v = Vector(math.cos(angle), math.sin(angle))
                self.field[i][j] = v

    def lookup(self, position):
        # Mencari vektor di posisi tertentu
        col = int(position.x // self.resolution)
        row = int(position.y // self.resolution)
        # Pastikan tidak keluar batas
        col = max(0, min(col, self.cols - 1))
        row = max(0, min(row, self.rows - 1))
        return self.field[col][row].copy()

    def display(self, canvas):
        # Visualisasi grid flow field
        for i in range(self.cols):
            for j in range(self.rows):
                x = i * self.resolution
                y = j * self.resolution
                v = self.field[i][j]
                # Gambar garis kecil menunjukkan arah aliran
                canvas.create_line(x, y, x + v.x * 10, y + v.y * 10, fill="gray")


# ========== Vehicle ==========
class Vehicle:
    def __init__(self, x, y, maxspeed, maxforce, width, height):
        self.position = Vector(x, y)  # Posisi awal
        self.velocity = Vector()  # Kecepatan
        self.acceleration = Vector()  # Percepatan
        self.maxspeed = maxspeed  # Kecepatan maksimum
        self.maxforce = maxforce  # Gaya maksimum
        self.r = 4  # Ukuran kendaraan
        self.w = width  # Lebar area
        self.h = height  # Tinggi area

    def apply_force(self, force):
        # Menerapkan gaya pada kendaraan
        self.acceleration.add(force)

    def follow(self, flowfield):
        # Mengikuti aliran flow field
        desired = flowfield.lookup(self.position)  # Dapatkan vektor aliran
        desired.mult(self.maxspeed)  # Skalakan dengan kecepatan maks
        steer = Vector.sub_vec(desired, self.velocity)  # Hitung gaya steering
        steer.limit(self.maxforce)  # Batasi gaya
        self.apply_force(steer)  # Terapkan gaya

    def update(self):
        # Update posisi kendaraan
        self.velocity.add(self.acceleration)
        self.velocity.limit(self.maxspeed)
        self.position.add(self.velocity)
        self.acceleration.mult(0)  # Reset percepatan

    def borders(self):
        # Membungkus kendaraan ke sisi lain jika keluar layar
        if self.position.x < -self.r:
            self.position.x = self.w + self.r
        if self.position.y < -self.r:
            self.position.y = self.h + self.r
        if self.position.x > self.w + self.r:
            self.position.x = -self.r
        if self.position.y > self.h + self.r:
            self.position.y = -self.r

    def display(self, canvas):
        # Gambar kendaraan sebagai segitiga
        theta = self.velocity.heading() + math.pi / 2  # Sudut menghadap
        x = self.position.x
        y = self.position.y
        r = self.r
        # Hitung 3 titik segitiga
        points = [
            (x + math.sin(theta) * -r * 2, y - math.cos(theta) * -r * 2),
            (x + math.sin(theta + math.pi * 2 / 3) * r * 2, y - math.cos(theta + math.pi * 2 / 3) * r * 2),
            (x + math.sin(theta - math.pi * 2 / 3) * r * 2, y - math.cos(theta - math.pi * 2 / 3) * r * 2)
        ]
        canvas.create_polygon(points, fill='lightblue', outline='black')


# ========== Main Program ==========
# Konfigurasi
WIDTH = 360  # Lebar canvas
HEIGHT = 240  # Tinggi canvas
debug = True  # Mode debug untuk menampilkan flow field

# Inisialisasi GUI
root = tk.Tk()
root.title("Flow Field Following")
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# Buat flow field dan kendaraan
flowfield = FlowField(20, WIDTH, HEIGHT)  # Grid 20x20
vehicles = [Vehicle(random.uniform(0, WIDTH), random.uniform(0, HEIGHT),
                    random.uniform(2, 5), random.uniform(0.1, 0.5), WIDTH, HEIGHT)
            for _ in range(80)]  # 80 kendaraan acak

def draw():
    canvas.delete("all")
    if debug:
        flowfield.display(canvas)  # Tampilkan flow field jika debug
    for v in vehicles:
        v.follow(flowfield)  # Kendaraan mengikuti aliran
        v.update()  # Update posisi
        v.borders()  # Cek batas layar
        v.display(canvas)  # Gambar kendaraan
    root.after(33, draw)  # Loop animasi (~30 fps)

draw()  # Mulai animasi
root.mainloop()  # Jalankan GUI

In [ ]:
import tkinter as tk
import random

# --- Kelas Vektor 2D ---
class Vector2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def add(self, other):
        self.x += other.x
        self.y += other.y

    def copy(self):
        return Vector2D(self.x, self.y)

# --- Setup Tkinter ---
window = tk.Tk()
window.title("Random Walker with Vectors + coords() + trail")
canvas = tk.Canvas(window, width=400, height=400, bg="white")
canvas.pack()

# --- Posisi awal walker ---
position = Vector2D(200, 200)
prev_position = position.copy()  # Simpan posisi sebelumnya

# --- Gambar titik awal (walker) dan teks posisi ---
walker = canvas.create_oval(position.x - 2, position.y - 2,
                            position.x + 2, position.y + 2,
                            fill="black")

info_text = canvas.create_text(200, 20, text="", font=("Arial", 10))

# --- Fungsi update langkah ---
def update():
    global prev_position

    # Simpan posisi sebelumnya
    prev_position = position.copy()

    # Gerakan acak: -1, 0, atau +1
    step = Vector2D(random.randint(-1, 1), random.randint(-1, 1))
    position.add(step)

    # Gambar garis jejak dari posisi sebelumnya ke posisi sekarang
    canvas.create_line(prev_position.x, prev_position.y,
                       position.x, position.y,
                       fill="gray")

    # Perbarui posisi titik walker
    canvas.coords(walker,
                  position.x - 2, position.y - 2,
                  position.x + 2, position.y + 2)

    # Update teks info posisi
    canvas.itemconfig(info_text,
                      text=f"Posisi: ({position.x}, {position.y})")

    # Panggil update lagi setelah 30 ms
    window.after(30, update)

# --- Jalankan animasi ---
update()
window.mainloop()
